In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('./cl2.csv')

In [4]:
from sklearn.model_selection import train_test_split

# splitting data
X_train, X_test, y_train, y_test = train_test_split(
                df.drop('target', axis=1),
                df['target'],
                test_size=0.2,
                random_state=42)

print("Shape of training set:", X_train.shape)
print("Shape of test set:", X_test.shape)

Shape of training set: (615, 14)
Shape of test set: (154, 14)


In [5]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
predictions1 = logreg.predict(X_test)

c:\Users\fafaf\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [6]:
from sklearn.metrics import confusion_matrix, classification_report

print("Confusion Matrix: \n", confusion_matrix(y_test, predictions1))
print('\n')
print(classification_report(y_test, predictions1))

Confusion Matrix: 
 [[64 17]
 [10 63]]


              precision    recall  f1-score   support

           0       0.86      0.79      0.83        81
           1       0.79      0.86      0.82        73

    accuracy                           0.82       154
   macro avg       0.83      0.83      0.82       154
weighted avg       0.83      0.82      0.82       154



In [11]:
model = LogisticRegression(warm_start=True,penalty='l2',max_iter=1000)

In [12]:
import utils
utils.set_initial_params(model, n_features=X_train.shape[1], n_classes=2)

In [13]:
import flwr as fl
import warnings
from sklearn.metrics import log_loss

class IrisClient(fl.client.NumPyClient):
        def get_parameters(self, config):  # type: ignore
            return utils.get_model_parameters(model)

        def fit(self, parameters, config):  # type: ignore
            utils.set_model_params(model, parameters)
            # Ignore convergence failure due to low local epochs
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                model.fit(X_train, y_train)
            accuracy = model.score(X_train, y_train)
            return (
                utils.get_model_parameters(model),
                len(X_train),
                {"train_accuracy": accuracy},
            )

        def evaluate(self, parameters, config):  # type: ignore
            utils.set_model_params(model, parameters)
            loss = log_loss(y_test, model.predict_proba(X_test))
            accuracy = model.score(X_test, y_test)
            return loss, len(X_test), {"test_accuracy": accuracy}

    # Start Flower client
fl.client.start_client(
    server_address="127.0.0.1:8080", client=IrisClient().to_client()
)

INFO flwr 2024-05-12 17:04:20,174 | grpc.py:52 | Opened insecure gRPC connection (no certificates were passed)
DEBUG flwr 2024-05-12 17:04:20,185 | connection.py:55 | ChannelConnectivity.IDLE
DEBUG flwr 2024-05-12 17:04:20,188 | connection.py:55 | ChannelConnectivity.READY
DEBUG flwr 2024-05-12 17:04:21,241 | connection.py:220 | gRPC channel closed
INFO flwr 2024-05-12 17:04:21,242 | app.py:398 | Disconnect and shut down


In [14]:
predictions1 = model.predict(X_test)
print(classification_report(y_test, predictions1))

              precision    recall  f1-score   support

           0       0.89      0.78      0.83        81
           1       0.78      0.89      0.83        73

    accuracy                           0.83       154
   macro avg       0.84      0.83      0.83       154
weighted avg       0.84      0.83      0.83       154



In [15]:
print("Confusion Matrix: \n", confusion_matrix(y_test, predictions1))

Confusion Matrix: 
 [[63 18]
 [ 8 65]]
